In [1]:
from tool.preprocess import Preprocess
from tool.s3_connect import S3_connector
from textrank import run_textrank
import pandas as pd
from wordToVector import WordToVector
from tqdm import tqdm
from database.db import *
from tqdm import tqdm
from scipy.spatial import distance
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
import numpy as np

In [2]:
s3_connector = S3_connector()
run = run_textrank.RunTextRank()
keys = run.key_extractor()
w = WordToVector()

In [188]:
data = select_not_null('정치', '2020-11-03')

In [189]:
len(data)

1215

In [190]:
vectors = []
key_sentences = []
keywords = []
indexs = []
for idx, key_sentence, keyword, date in data:
    try:
        if type(w.vectorize_word_list(keyword[1:-1].split(","))) is float:
            continue
        indexs.append(idx)
        key_sentences.append(key_sentence[1:-1].split(","))
        keywords.append(keyword[1:-1].split(","))
        w_vector = (w.vectorize_word_list(keyword[1:-1].split(","))).tolist()
        vectors.append(w_vector)
    except:
        continue
    

In [191]:
print(len(vectors))
print(len(key_sentences))
print(len(keywords))

1212
1212
1212


In [192]:
# headline = list(map(lambda x : [x[1]], headline))

In [193]:
df = pd.DataFrame({'news_id':indexs,'key_sentence':key_sentences,'keywords':keywords,'vectors':vectors})
df.head()

,news_id,key_sentence,keywords,vectors
0,100-003-0010164328,"[""11월4일 점자의날 기념 학생들과 점자대회 참여 건넨 카드에는 점자로 꿈 닿지 ...","[점자, 서울, 여사, 학생, 인근, 장애, 대회, 기념]","[-0.0552370585501194, 0.20425309240818024, 0.0..."
1,100-016-0001746642,"[""송 의원이 발의한 개정안 역시 정부의 공시가격 상향 등 부동산 세제 강화로 야기...","[주택, 재산세, 국민, 부동산, 가격, 감면, 최대, 의원, 공시]","[-0.04012050852179527, 0.2543265223503113, 0.7..."
2,100-001-0011989353,"[""공화당 도널드 트럼프 대통령과 민주당 조 바이든 대선 후보의 운명을 가를 투표가...","[투표, 후보, 대선, 대통령, 바이든, 트럼프, 미국]","[0.34906312823295593, 0.09188582003116608, 0.5..."
3,100-032-0003041097,"[""경향신문 더불어민주당 국회 외교통일위원회 소속 의원들과 정부는 3일 당·정 협의...","[대선, 미국, 국회, 장관, 통일부, 적극, 이후, 민주당, 의원, 외교부]","[0.7968959212303162, 0.13078045845031738, 1.08..."
4,100-001-0011989361,"[""만일 트럼프가 펜실베이니아를 빼앗긴다면 선벨트 싹쓸이에 더해 미시간 또는 위스콘...","[트럼프, 펜실베이니아, 바이든, 포인트, 대선, 선거인단, 시간, 플로리다, 연합뉴스]","[-0.009733825922012329, -0.26111990213394165, ..."


In [194]:
len(df)

1212

In [195]:
# ### cosine similarity ###
# cosine = {}
# for i in range(2,len(df)):
#      cosine[i] = 1 - distance.cosine(df['vectors'][1], df['vectors'][i])
# cosine_new = list(cosine.items())
# cosine_new.sort(key=lambda x:x[1], reverse=True)
# print(f"base : {df['key_sentence'][1]}")
# for i,j in cosine_new:
#     print(f"{i} {df['key_sentence'][i]}  | {j}")
#     print('')

In [175]:
# values = [i[1] for i in cosine_new]
# plt.figure(figsize=(20,10))
# plt.plot([i for i in range(len(values))][0:200], values[0:200])

In [201]:
# for i in range(len(df['vectors'])):
#     if type(df['vectors'][i]) == float:
#             print(df.iloc[i])

In [84]:
# db = DBSCAN(eps=0.0011, min_samples=3, metric='cosine', algorithm='brute').fit(vectors)

In [85]:
# core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
# core_samples_mask[db.core_sample_indices_] = True
# labels = db.labels_

In [91]:
# labels

array([-1, -1, -1, ..., -1, -1, -1])

In [196]:
import time

In [197]:
def clustering(df, pivot):
    start_time = time.time()
    cosine_dict = {}
    search = {}
    for i_index in tqdm(range(len(df))):
        start_time = time.time()
        stack = []
        result = {}
        if i_index in search:
            continue
        for j_index in range(len(df)):
            if i_index == j_index:
                continue
            if j_index in cosine_dict:
                continue
            if j_index in search:
                continue
            cosine = 1 - distance.cosine(df['vectors'][i_index], df['vectors'][j_index])
            if cosine <= pivot:
                continue
            stack.append(j_index)
            
        while len(stack) != 0:
            index_value = stack.pop()
            for k_index in range(len(df)):
                
                if index_value == k_index:
                    continue
                    
                if k_index in stack:
                    continue
                    
                if i_index == k_index:
                    continue
                    
                if k_index in result:
                    continue
                    
                if k_index in cosine_dict:
                    continue
                    
                if k_index in search:
                    continue
                    
                cosine = 1 - distance.cosine(df['vectors'][index_value], df['vectors'][k_index])
                if cosine <= pivot:
                    continue
                result.update({k_index: 0})
                search.update({k_index: 0})
            result.update({index_value: 0})
            search.update({index_value: 0})
                
        cosine_dict[i_index] = list(result.keys())
        
    return cosine_dict

In [204]:
clustered = clustering(df, 0.9)

100%|██████████| 1212/1212 [00:33<00:00, 36.19it/s] 


In [207]:
def find_representative(clustered):
    representatives = []
    for key,value in clustered.items():
        clustered[key].append(key)
        sums = np.array([0.0]*300)
        for i in clustered[key]:
            sums += np.array(df['vectors'][i])
        check = sums/len(clustered[key])
        cosine = {}
        for j in clustered[key]:
            cosine[j] = 1- distance.cosine(check, df['vectors'][j])
        cosine_new = list(cosine.items())
        cosine_new.sort(key=lambda x:x[1],reverse=True)
        representatives.append(df['news_id'][cosine_new[0][0]])
    return representatives

In [208]:
representatives = find_representative(clustered)

In [209]:
len(representatives)

620

In [212]:
representatives

['100-003-0010164328',
 '100-001-0011989962',
 '100-001-0011989353',
 '100-421-0004968486',
 '100-001-0011989361',
 '100-001-0011989360',
 '100-449-0000199828',
 '100-003-0010165296',
 '100-047-0002290402',
 '100-011-0003820226',
 '100-016-0001746573',
 '100-001-0011988781',
 '100-047-0002290393',
 '100-015-0004442639',
 '100-001-0011989290',
 '100-014-0004521768',
 '100-018-0004776261',
 '100-008-0004494137',
 '100-449-0000199825',
 '100-015-0004442634',
 '100-469-0000550307',
 '100-011-0003820192',
 '100-025-0003049095',
 '100-003-0010164160',
 '100-023-0003573370',
 '100-022-0003519343',
 '100-214-0001077086',
 '100-422-0000455048',
 '100-001-0011989125',
 '100-018-0004776244',
 '100-022-0003519520',
 '100-008-0004494266',
 '100-081-0003136431',
 '100-015-0004442596',
 '100-001-0011989067',
 '100-417-0000613867',
 '100-031-0000566264',
 '100-021-0002448016',
 '100-421-0004967164',
 '100-277-0004784887',
 '100-310-0000081615',
 '100-088-0000670223',
 '100-001-0011989036',
 '100-032-0

In [213]:
from tool.preprocess import Preprocess
from tool.s3_connect import S3_connector
from textrank import run_textrank
import pandas as pd
from wordToVector import WordToVector
from tqdm import tqdm
from database.db import *
from clustering import *


In [214]:
s3_connector = S3_connector()

In [215]:
bucket = s3_connector.get_bucket()
bucket

s3.Bucket(name='newsdesk-raw')

In [217]:
array = s3_connector.get_all_newsdata_key()

In [229]:
for item in array:
    category = item[:-15]
    date = item[-14:-4]

경제
2020-09-01
경제
2020-09-02
경제
2020-09-03
경제
2020-09-04
경제
2020-09-05
경제
2020-09-06
경제
2020-09-07
경제
2020-09-08
경제
2020-09-09
경제
2020-09-10
경제
2020-09-11
경제
2020-09-12
경제
2020-09-13
경제
2020-09-14
경제
2020-09-15
경제
2020-09-16
경제
2020-09-17
경제
2020-09-18
경제
2020-09-19
경제
2020-09-20
경제
2020-09-21
경제
2020-09-22
경제
2020-09-23
경제
2020-09-24
경제
2020-09-25
경제
2020-09-26
경제
2020-09-27
경제
2020-09-28
경제
2020-09-29
경제
2020-09-30
경제
2020-10-01
경제
2020-10-02
경제
2020-10-03
경제
2020-10-04
경제
2020-10-05
경제
2020-10-06
경제
2020-10-07
경제
2020-10-08
경제
2020-10-09
경제
2020-10-10
경제
2020-10-11
경제
2020-10-12
경제
2020-10-13
경제
2020-10-14
경제
2020-10-15
경제
2020-10-16
경제
2020-10-17
경제
2020-10-18
경제
2020-10-19
경제
2020-10-20
경제
2020-10-21
경제
2020-10-22
경제
2020-10-23
경제
2020-10-24
경제
2020-10-25
경제
2020-10-26
경제
2020-10-27
경제
2020-10-28
경제
2020-10-29
경제
2020-10-30
경제
2020-10-31
경제
2020-11-01
경제
2020-11-02
경제
2020-11-03
경제
2020-11-04
경제
2020-11-05
경제
2020-11-06
경제
2020-11-07
경제
2020-11-08
경제
2020-11-09
경제
2020-11-10
경제
202